In [1]:
!pip install scikit-learn
!pip install pandas 
!pip install datasets 
!pip install transformers

In [2]:
from datasets import load_dataset


dataset = load_dataset("cbt", "CN")

Generating train split:   0%|          | 0/120769 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset cbt downloaded and prepared to /root/.cache/huggingface/datasets/cbt/CN/1.1.0/73e4c9316b0d86a7addd7f80183fb971a6161fa2f8b746da034e205b7e16f78d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_list_proba = [dataset['test'][0]]
data_list = dataset['test']

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np 
import torch 



model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

d = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}

model.parallelize(d)

In [5]:
import copy 

def score(text_before, option, text_after):
    
    merged = text_before + option + text_after
    tmp = tokenizer.encode( text_before, add_special_tokens=False, return_tensors="pt").cuda()
    text_tokenized = tokenizer.encode( merged, add_special_tokens=False, return_tensors="pt").cuda()
    
    lnn_tmp = tmp.shape[1]
    lnn = text_tokenized.shape[1]
    
    labels_ = copy.deepcopy(text_tokenized)
    labels_[0][:(lnn_tmp-1)] = -100
    #print('lnn je ', lnn)
    #print('lnn_tmp je ', lnn_tmp)
    #print(labels_)
    #print(merged)
    #print(lnn)
    #print((text_tokenized))
    
    if lnn > 1024:
        return -1000000
    try:
        model_output = model(text_tokenized, labels = labels_)
        logits = model_output.logits
        #print('model _output je ', 'logits je ', logits)
        #print(logits.shape)
        #print(model_output.loss.item())
        return model_output.loss.item()

    except:
        print(len(text_tokenized))
        print((text_tokenized.shape)[1])
        print(text_before_tokenized)
#        exit(1)


In [6]:
nmb = 0
correct = 0
for test in data_list:
    nmb +=1
    if nmb == 195:
        continue
    sentence = test['sentences']
    answer = test['question']
    options = test['options']
    answers = []
    sentence_options = []
    tmp = answer.split('XXXXX')
    answer_before = tmp[0]
    answer_after = tmp[1]
    for i in options:
        #' '.join(sentence) + 
        changed = answer.replace('XXXXX', i)
        sentence_options.append(changed)
        answers.append(i)
    
    model_answer = ''
    min_loss = 1 * float('inf')
    
    for i in range(len(sentence_options)):        
        scr = score(' '.join(sentence) + answer_before, options[i], answer_after)
        #print("scr je ", scr)
        if scr < min_loss:
            min_loss = scr
            model_answered = answers[i]
            
    if model_answered == test['answer']:
        correct += 1
        print('CORRECT')
    
    else:
        print('INCORRECT')
        
    if nmb % 50 == 0:
        print(correct / nmb)
    #print(scores)
    #print(answers)

print(correct / nmb)

CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
0.8
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
0.82
INCORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT

Token indices sequence length is longer than the specified maximum sequence length for this model (1120 > 1024). Running this sequence through the model will result in indexing errors


INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
0.8057142857142857
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
0.8175
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
CORRECT
INCORRECT
CORRECT
